In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 2.0/2.0 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 4.9 MB/s eta 0:00:00
     --------------------------------------- 46.2/46.2 MB 11.7 MB/s eta 0:00:00
     --------------------------------------- 11.0/11.0 MB 14.5 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 18.2 MB/s eta 0:00:00
     -------------------------------------- 103.2/103.2 kB 5.8 MB/s eta 0:00:00
     ------------------------------------- 159.8/159.8 MB 34.5 MB/s eta 0:00:00
     ---------------------------------------- 9.9/9.9 MB 31.5 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     ------------------------------------- 301.8/301.8 kB 18.2 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd

DEMO_URL = 'http://ctgan-demo.s3.amazonaws.com/census.csv.gz'


def load_demo():
    """Load the demo."""
    return pd.read_csv(DEMO_URL, compression='gzip')

df = load_demo()
# adult_label = adult.iloc[:,-1]
# target = adult.iloc[:,:-1]
print(df.columns)
df

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
from sklearn.preprocessing import KBinsDiscretizer

# Assuming 'Feature1', 'Feature2', 'Feature3', and 'Feature4' are continuous variables
features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Discretize the continuous features into 3 bins each
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
df[features] = discretizer.fit_transform(df[features])

print(df.head())


   age          workclass  fnlwgt   education  education-num  \
0  0.0          State-gov     0.0   Bachelors            2.0   
1  1.0   Self-emp-not-inc     0.0   Bachelors            2.0   
2  0.0            Private     0.0     HS-grad            1.0   
3  1.0            Private     0.0        11th            1.0   
4  0.0            Private     0.0   Bachelors            2.0   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0           0.0           0.0             1.0   United-States   <=50

In [5]:
from pgmpy.models import BayesianNetwork

# Define the structure of the Bayesian Network
# Assume the target variable is 'Target' and the features are 'Feature1', 'Feature2', 'Feature3', and 'Feature4'
model = BayesianNetwork([('age', 'income'),
                         ('workclass', 'income'),
                         ('fnlwgt', 'income'),
                         ('education', 'income'),
                         ('education-num', 'income'),
                         ('marital-status', 'income'),
                        ('occupation', 'income'),
                         ('relationship', 'income'),
                         ('race', 'income'),
                           ('sex', 'income'),
                         ('capital-gain', 'income'),
                         ('capital-loss', 'income'),
                         ('hours-per-week', 'income'),
                         ('native-country', 'income'),
                         ]
                        )


D:\SummerResearch\summer_research\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pgmpy.estimators import MaximumLikelihoodEstimator

# Fit the model to the data
model.fit(df, estimator=MaximumLikelihoodEstimator)


MemoryError: Unable to allocate 25.9 GiB for an array with shape (27776649600,) and data type int8

In [ ]:
from pgmpy.inference import VariableElimination

# Create an inference object
inference = VariableElimination(model)

# Assume we have a new data point with the features
new_data = pd.read_csv("adult_sdv_100k_onlyX.csv")

# Predict the target
predicted_target = inference.map_query(variables=['Target'], evidence=new_data)
print(predicted_target)
